In [1]:
import os

In [7]:
import json

input_txt = "captions.txt"     # 你的原始txt文件名
output_jsonl = "captions.jsonl"  # 输出的jsonl文件名

with open(input_txt, "r", encoding="utf-8") as fin, open(output_jsonl, "w", encoding="utf-8") as fout:
    for line in fin:
        line = line.strip()
        if not line:
            continue
        # 拆分为文件名和描述
        if ':' in line:
            image_name, caption = line.split(':', 1)
            image_name = image_name.strip()
            caption = caption.strip()
            # 组装成json并写入
            json.dump({"image": image_name, "caption": caption}, fout, ensure_ascii=False)
            fout.write("\n")


In [ ]:
pip install torch torchvision peft

In [3]:
import torch
import transformers
# import peft
print(torch.__version__)
print(transformers.__version__)
# print(peft.__version__)


2.7.1+cpu
4.40.2


## 划分训练集验证集

In [8]:
import json
import random

input_file = "C:/Users/taste/Documents/0_sis/captions.jsonl"
train_file = "C:/Users/taste/Documents/0_sis/captions_train.jsonl"
val_file = "C:/Users/taste/Documents/0_sis/captions_val.jsonl"

# 读入所有数据
with open(input_file, "r", encoding="utf-8") as fin:
    data = [json.loads(line) for line in fin if line.strip()]

# 打乱顺序
random.shuffle(data)

# 按8:2分
split_idx = int(len(data) * 0.8)
train_data = data[:split_idx]
val_data = data[split_idx:]

# 写入新文件
with open(train_file, "w", encoding="utf-8") as ftrain:
    for item in train_data:
        ftrain.write(json.dumps(item, ensure_ascii=False) + "\n")

with open(val_file, "w", encoding="utf-8") as fval:
    for item in val_data:
        fval.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"训练集写入 {train_file}，共 {len(train_data)} 条")
print(f"验证集写入 {val_file}，共 {len(val_data)} 条")


训练集写入 C:/Users/taste/Documents/0_sis/captions_train.jsonl，共 160 条
验证集写入 C:/Users/taste/Documents/0_sis/captions_val.jsonl，共 40 条


## 开始微调

### 第一步，加载模型

In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
from tqdm import tqdm

c:\Users\taste\Documents\0_sis\.florence2env2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# 加载假flash_attn
import sys, types, importlib.machinery

def make_fake_mod(name):
    mod = types.ModuleType(name)
    mod.__spec__ = importlib.machinery.ModuleSpec(name, None)
    return mod

sys.modules["flash_attn"] = make_fake_mod("flash_attn")
sys.modules["flash_attn.ops"] = make_fake_mod("flash_attn.ops")
sys.modules["flash_attn.bert_flash_attention"] = make_fake_mod("flash_attn.bert_flash_attention")

In [3]:
import json
import os
from torch.utils.data import Dataset
from PIL import Image
import csv
from peft import LoraConfig, get_peft_model

In [4]:
class Florence2CaptionDataset(Dataset):
    def __init__(self, jsonl_path, image_dir):
        self.samples = []
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                if line.strip():
                    item = json.loads(line)
                    self.samples.append(item)
        self.image_dir = image_dir

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        item = self.samples[idx]
        image_path = os.path.join(self.image_dir, item["image"])
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        prompt = "<DETAILED_CAPTION>"
        caption = item["caption"]
        return {"prompt": prompt, "caption": caption, "image": image}

训练从这开始

In [5]:
# 加载florence-2-base模型
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

CKPT = "microsoft/Florence-2-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    CKPT, trust_remote_code=True
).to(DEVICE)

processor = AutoProcessor.from_pretrained(
    CKPT, trust_remote_code=True
)

c:\Users\taste\Documents\0_sis\.florence2env2\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
# 简化版本
def collate_fn(batch):
    prompts = [b["prompt"] for b in batch]
    captions = [b["caption"] for b in batch]
    images = [b["image"] for b in batch]
    model_inputs = processor(
        text=prompts,
        images=images,
        return_tensors="pt",
        padding=True
    )
    labels = processor(
        text=captions,
        images=images,
        return_tensors="pt",
        padding=True
    ).input_ids
    model_inputs["labels"] = labels
    return {k: v.to(model.device) for k, v in model_inputs.items()}


In [7]:
# 加载数据
train_jsonl = r"C:\Users\taste\Documents\0_sis\captions_train.jsonl"
val_jsonl = r"C:\Users\taste\Documents\0_sis\captions_val.jsonl"
img_dir = r"C:\Users\taste\Documents\0_sis\processed"

train_ds = Florence2CaptionDataset(train_jsonl, img_dir)
val_ds = Florence2CaptionDataset(val_jsonl, img_dir)

In [8]:
# ==== 1. 参数网格 ====
param_grid = [
    #{"BATCH_SIZE": 4, "LEARNING_RATE": 5e-6, "EPOCHS": 15},
    #{"BATCH_SIZE": 4, "LEARNING_RATE": 2e-5, "EPOCHS": 15},
    #{"BATCH_SIZE": 8, "LEARNING_RATE": 5e-6, "EPOCHS": 15},
    #{"BATCH_SIZE": 2, "LEARNING_RATE": 1e-5, "EPOCHS": 15},
    {"BATCH_SIZE": 4, "LEARNING_RATE": 1e-5, "EPOCHS": 30},
]

# ==== 2. LoRA 配置 ====
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "linear", "Conv2d", "lm_head", "fc2"],
    task_type="CAUSAL_LM",
    lora_dropout=0.1,
    bias="none"
)

# ==== 3. 推理函数（每组参数后可以用来评估效果）====
def infer_caption(model, processor, img_path, device):
    from PIL import Image
    img = Image.open(img_path).convert("RGB")
    prompt = "<DETAILED_CAPTION>"
    inputs = processor(text=prompt, images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs.input_ids,
            pixel_values=inputs.pixel_values,
            max_new_tokens=80,
            num_beams=3
        )
    output = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f"图片 {img_path} 生成描述：{output}")

In [9]:
# ==== 4. 初始化CSV ====
loss_csv_path = "train_val_loss_log_5.csv"
with open(loss_csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["param_group", "epoch", "train_loss", "val_loss"])

### 正式开始循环训练

In [10]:
for params in param_grid:
    BATCH_SIZE = params["BATCH_SIZE"]
    LEARNING_RATE = params["LEARNING_RATE"]
    EPOCHS = params["EPOCHS"]
    LOG_STEP = 10

    print(f"\n===== 实验参数: BATCH_SIZE={BATCH_SIZE}, LR={LEARNING_RATE}, EPOCHS={EPOCHS} =====")
    # 参数组唯一目录
    param_name = f"bs{BATCH_SIZE}_lr{LEARNING_RATE:.0e}_ep{EPOCHS}"
    # 每组参数专属文件夹
    param_dir = f"./florence2-lora-{param_name}"
    os.makedirs(param_dir, exist_ok=True)

    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    # 重新加载基础模型与 LoRA
    base_model = AutoModelForCausalLM.from_pretrained(CKPT, trust_remote_code=True).to(DEVICE)
    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()

    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    num_training_steps = EPOCHS * len(train_dl)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    best_val_loss = float("inf")
    best_ckpt_dir = ""
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        batch_losses = []
        progress_bar = tqdm(enumerate(train_dl), total=len(train_dl), desc=f"Exp[{BATCH_SIZE},{LEARNING_RATE}] Epoch {epoch+1} Train")
        for step, batch in progress_bar:
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            total_loss += loss.item()
            batch_losses.append(loss.item())
            if (step + 1) % LOG_STEP == 0 or (step + 1) == len(train_dl):
                avg_loss = sum(batch_losses[-LOG_STEP:]) / min(LOG_STEP, len(batch_losses[-LOG_STEP:]))
                progress_bar.set_postfix({"avg_loss": avg_loss})
        avg_epoch_loss = total_loss / len(train_dl)
        print(f"Epoch {epoch+1} - Train avg loss: {avg_epoch_loss:.4f}")

        # 验证
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_dl, desc=f"Epoch {epoch+1} Val"):
                outputs = model(**batch)
                val_loss += outputs.loss.item()
        avg_val_loss = val_loss / len(val_dl)
        print(f"Epoch {epoch+1} - Val avg loss: {avg_val_loss:.4f}")

        # ==== 记录到 CSV ====
        with open(loss_csv_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([param_name, epoch+1, avg_epoch_loss, avg_val_loss])

        # === 保存每一轮 epoch checkpoint（子文件夹）===
        epoch_dir = os.path.join(param_dir, f"epoch{epoch+1:02d}")
        os.makedirs(epoch_dir, exist_ok=True)
        print(f"保存本轮模型: {epoch_dir}")
        model.save_pretrained(epoch_dir)
        processor.save_pretrained(epoch_dir)

        # === 保存 best checkpoint（每组 best 子文件夹）===
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_ckpt_dir = os.path.join(param_dir, "best")
            model.save_pretrained(best_ckpt_dir)
            processor.save_pretrained(best_ckpt_dir)
            print(f"保存最优模型: {best_ckpt_dir}")

    print(f"\n>>> 参数组 [{param_name}] 训练完成,最佳验证集loss: {best_val_loss:.4f}，最优模型保存在: {best_ckpt_dir}")

    # ==== 推理评估（可选）====
    infer_caption(model, processor, r"C:\Users\taste\Documents\0_sis\processed\image_145.jpg", DEVICE)



===== 实验参数: BATCH_SIZE=4, LR=1e-05, EPOCHS=30 =====


c:\Users\taste\Documents\0_sis\.florence2env2\lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


trainable params: 1,929,928 || all params: 233,343,944 || trainable%: 0.8270743893829102


Exp[4,1e-05] Epoch 1 Train: 100%|██████████| 40/40 [10:16<00:00, 15.40s/it, avg_loss=5.51]


Epoch 1 - Train avg loss: 5.6013


...

Epoch 29 - Val avg loss: 1.9443
保存本轮模型: ./florence2-lora-bs4_lr1e-05_ep30\epoch29
保存最优模型: ./florence2-lora-bs4_lr1e-05_ep30\best


Exp[4,1e-05] Epoch 30 Train: 100%|██████████| 40/40 [09:33<00:00, 14.35s/it, avg_loss=1.79]


Epoch 30 - Train avg loss: 1.9323


Epoch 30 Val: 100%|██████████| 10/10 [00:33<00:00,  3.33s/it]


Epoch 30 - Val avg loss: 1.9437
保存本轮模型: ./florence2-lora-bs4_lr1e-05_ep30\epoch30
保存最优模型: ./florence2-lora-bs4_lr1e-05_ep30\best

>>> 参数组 [bs4_lr1e-05_ep30] 训练完成,最佳验证集loss: 1.9437，最优模型保存在: ./florence2-lora-bs4_lr1e-05_ep30\best
图片 C:\Users\taste\Documents\0_sis\processed\image_145.jpg 生成描述：A store front with a large sign reading "Pelquuería Murano." The glass facade reflects the modern design of the store.


## 加训

In [ ]:
# 加载假flash_attn
import sys, types, importlib.machinery

def make_fake_mod(name):
    mod = types.ModuleType(name)
    mod.__spec__ = importlib.machinery.ModuleSpec(name, None)
    return mod

sys.modules["flash_attn"] = make_fake_mod("flash_attn")
sys.modules["flash_attn.ops"] = make_fake_mod("flash_attn.ops")
sys.modules["flash_attn.bert_flash_attention"] = make_fake_mod("flash_attn.bert_flash_attention")

In [2]:
import os
import json
import csv
from PIL import Image
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, AutoModelForCausalLM, AdamW, get_scheduler
from peft import PeftModel

# === 数据类 ===
class Florence2CaptionDataset(Dataset):
    def __init__(self, jsonl_path, image_dir):
        self.samples = []
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                if line.strip():
                    item = json.loads(line)
                    self.samples.append(item)
        self.image_dir = image_dir

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        item = self.samples[idx]
        image_path = os.path.join(self.image_dir, item["image"])
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        prompt = "<DETAILED_CAPTION>"
        caption = item["caption"]
        return {"prompt": prompt, "caption": caption, "image": image}

# === collate_fn ===
def collate_fn(batch):
    prompts = [b["prompt"] for b in batch]
    captions = [b["caption"] for b in batch]
    images = [b["image"] for b in batch]
    model_inputs = processor(
        text=prompts,
        images=images,
        return_tensors="pt",
        padding=True
    )
    labels = processor(
        text=captions,
        images=images,
        return_tensors="pt",
        padding=True
    ).input_ids
    model_inputs["labels"] = labels
    return {k: v.to(model.device) for k, v in model_inputs.items()}

c:\Users\taste\Documents\0_sis\.florence2env2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# === 推理函数（可选） ===
def infer_caption(model, processor, img_path, device):
    img = Image.open(img_path).convert("RGB")
    prompt = "<DETAILED_CAPTION>"
    inputs = processor(text=prompt, images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs.input_ids,
            pixel_values=inputs.pixel_values,
            max_new_tokens=80,
            num_beams=3
        )
    output = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f"图片 {img_path} 生成描述：{output}")


In [4]:
# === 加载数据 ===
train_jsonl = r"C:\Users\taste\Documents\0_sis\captions_train.jsonl"
val_jsonl = r"C:\Users\taste\Documents\0_sis\captions_val.jsonl"
img_dir = r"C:\Users\taste\Documents\0_sis\processed"

train_ds = Florence2CaptionDataset(train_jsonl, img_dir)
val_ds = Florence2CaptionDataset(val_jsonl, img_dir)

In [5]:
# === 训练配置 ===
CKPT = "microsoft/Florence-2-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
resume_ckpt_dir = "./florence2-lora-bs4_lr1e-05_ep30/best"  # 你的最佳模型路径

BATCH_SIZE = 4
LEARNING_RATE = 5e-6
EPOCHS = 15  # 加训轮数
LOG_STEP = 10

param_name = "bs4_lr1e-5_ep30_plus15"
param_dir = f"./florence2-lora-{param_name}"
os.makedirs(param_dir, exist_ok=True)

In [6]:
# === 加载模型 ===
#base_model = AutoModelForCausalLM.from_pretrained(CKPT, trust_remote_code=True).to(DEVICE)
#model = PeftModel.from_pretrained(base_model, resume_ckpt_dir).to(DEVICE)
#model.print_trainable_parameters()
from peft import PeftModel, PeftConfig

# === 正确加载 LoRA 的方式 ===
print("📥 正在加载 LoRA 配置...")
peft_config = PeftConfig.from_pretrained(resume_ckpt_dir)

print("📥 加载 base 模型...")
base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path, trust_remote_code=True)

print("📥 注入 LoRA...")
model = PeftModel.from_pretrained(base_model, resume_ckpt_dir)

# === 转为 GPU 并设置训练模式 ===
model = model.to(DEVICE)
model.train()

# === 确保所有参数是可训练的（尤其是 LoRA 注入后）===
for param in model.parameters():
    param.requires_grad = True

# === 检查 trainable 参数数量 ===
trainable_params = sum(p.numel() for p in model.parameters() if param.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"✅ Trainable params: {trainable_params} / {total_params} ({100*trainable_params/total_params:.2f}%)")


# === 加载 Processor ===
processor = AutoProcessor.from_pretrained(resume_ckpt_dir, trust_remote_code=True)

# === DataLoader ===
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# === 优化器 & Scheduler ===
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = EPOCHS * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# === 初始化日志文件 ===
loss_csv_path = f"train_val_loss_log_{param_name}.csv"
with open(loss_csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["param_group", "epoch", "train_loss", "val_loss"])

📥 正在加载 LoRA 配置...
📥 加载 base 模型...


c:\Users\taste\Documents\0_sis\.florence2env2\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


📥 注入 LoRA...
✅ Trainable params: 233343944 / 233343944 (100.00%)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\taste\Documents\0_sis\.florence2env2\lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
# === 开始训练 ===
best_val_loss = float("inf")
best_ckpt_dir = ""
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    batch_losses = []
    progress_bar = tqdm(enumerate(train_dl), total=len(train_dl), desc=f"ContinueTrain Epoch {epoch+1}")
    for step, batch in progress_bar:
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        batch_losses.append(loss.item())
        if (step + 1) % LOG_STEP == 0 or (step + 1) == len(train_dl):
            avg_loss = sum(batch_losses[-LOG_STEP:]) / min(LOG_STEP, len(batch_losses[-LOG_STEP:]))
            progress_bar.set_postfix({"avg_loss": avg_loss})
    avg_epoch_loss = total_loss / len(train_dl)
    print(f"Epoch {epoch+1} - Train avg loss: {avg_epoch_loss:.4f}")

    # === 验证 ===
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_dl, desc=f"Epoch {epoch+1} Val"):
            outputs = model(**batch)
            val_loss += outputs.loss.item()
    avg_val_loss = val_loss / len(val_dl)
    print(f"Epoch {epoch+1} - Val avg loss: {avg_val_loss:.4f}")

    # === 记录 loss ===
    with open(loss_csv_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([param_name, epoch + 1, avg_epoch_loss, avg_val_loss])

    # === 保存每轮 checkpoint ===
    epoch_dir = os.path.join(param_dir, f"epoch{epoch+1:02d}")
    os.makedirs(epoch_dir, exist_ok=True)
    print(f"保存本轮模型: {epoch_dir}")
    model.save_pretrained(epoch_dir)
    processor.save_pretrained(epoch_dir)

    # === 保存最优 checkpoint ===
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_ckpt_dir = os.path.join(param_dir, "best")
        model.save_pretrained(best_ckpt_dir)
        processor.save_pretrained(best_ckpt_dir)
        print(f"保存最优模型: {best_ckpt_dir}")

print(f"\n>>> 加训完成, 最佳验证集 loss: {best_val_loss:.4f}，模型保存在: {best_ckpt_dir}")

# === 选做：生成一个样本进行推理 ===
infer_caption(model, processor, r"C:\Users\taste\Documents\0_sis\processed\image_145.jpg", DEVICE)

ContinueTrain Epoch 1: 100%|██████████| 40/40 [12:27<00:00, 18.69s/it, avg_loss=1.93]


Epoch 1 - Train avg loss: 1.8489


Epoch 1 Val: 100%|██████████| 10/10 [00:33<00:00,  3.34s/it]
c:\Users\taste\Documents\0_sis\.florence2env2\lib\site-packages\peft\utils\save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


Epoch 1 - Val avg loss: 1.7784
保存本轮模型: ./florence2-lora-bs4_lr1e-5_ep30_plus15\epoch01
保存最优模型: ./florence2-lora-bs4_lr1e-5_ep30_plus15\best


ContinueTrain Epoch 2: 100%|██████████| 40/40 [12:29<00:00, 18.73s/it, avg_loss=1.53]


...

Epoch 14 - Val avg loss: 1.7283
保存本轮模型: ./florence2-lora-bs4_lr1e-5_ep30_plus15\epoch14


ContinueTrain Epoch 15: 100%|██████████| 40/40 [12:42<00:00, 19.06s/it, avg_loss=0.537]


Epoch 15 - Train avg loss: 0.5249


Epoch 15 Val: 100%|██████████| 10/10 [00:33<00:00,  3.39s/it]


Epoch 15 - Val avg loss: 1.7299
保存本轮模型: ./florence2-lora-bs4_lr1e-5_ep30_plus15\epoch15

>>> 加训完成, 最佳验证集 loss: 1.6491，模型保存在: ./florence2-lora-bs4_lr1e-5_ep30_plus15\best
图片 C:\Users\taste\Documents\0_sis\processed\image_145.jpg 生成描述：A modern shopfront with large glass windows and a white sign reading "Pelquería Murano." The signage and branding indicate a motorcycle repair or replacement shop.


In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM

# 路径改为你的最新微调目录
ckpt_dir = "./florence2-lora-epoch5"

model = AutoModelForCausalLM.from_pretrained(ckpt_dir, trust_remote_code=True).to(DEVICE)
processor = AutoProcessor.from_pretrained(ckpt_dir, trust_remote_code=True)
model.eval()

In [9]:
from PIL import Image

img_path = r"C:\Users\taste\Documents\0_sis\processed\image_181.jpg"  # 换成你想测试的图片名
img = Image.open(img_path).convert("RGB")
prompt = "<DETAILED_CAPTION>"

inputs = processor(text=prompt, images=img, return_tensors="pt").to(model.device)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=inputs.input_ids,
        pixel_values=inputs.pixel_values,
        max_new_tokens=80,
        num_beams=3  # 可以试试改为5、8，结果可能更好
    )
output = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("生成的描述：", output)

生成的描述： The image shows a person walking past a Louis Barton store in London. The store has glass doors and a board with text on it.


In [10]:
img_path = r"C:\Users\taste\Documents\0_sis\processed\image_145.jpg"  # 换成你想测试的图片名
img = Image.open(img_path).convert("RGB")
prompt = "<DETAILED_CAPTION>"

inputs = processor(text=prompt, images=img, return_tensors="pt").to(model.device)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=inputs.input_ids,
        pixel_values=inputs.pixel_values,
        max_new_tokens=80,
        num_beams=3  # 可以试试改为5、8，结果可能更好
    )
output = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("生成的描述：", output)

生成的描述： The image shows a store front with a sign that reads "Murano" and a bicycle parked in front of it. There is a person sitting on a chair in the foreground, and a few plants in the background. The image is slightly blurred, giving it a dreamy quality.
